# Group spectral-peak detections on all geophone stations by time and frequency

In [ ]:
# Imports
from os.path import join
from numpy import linspace
from pandas import read_csv, Grouper, cut

from utils_basic import SPECTROGRAM_DIR as indir, GEO_STATIONS as stations
from utils_spec import assemble_spec_filename, read_geo_spectrograms, find_geo_station_spectral_peaks
from utils_plot import plot_geo_total_psd_and_peaks, save_figure

In [ ]:
# Inputs
# Data
days = ["20200113"]
window_length = 60.0
overlap = 0.0
downsample = True
downsample_factor = 60

# Grouping
time_bin_width = 1.0 # in minutes
freq_bin_width = 1.0 # in Hz
minfreq = 0.0
maxfreq = 500.0

# Directory
indir = join(indir, "daily")

# Plotting
day_to_plot = "20200113"

In [ ]:
# Read the CSV files containing the spectral peaks
for day in days:
    print(f"Processing day {day}")

    # Read the spectrograms
    filename = f"daily_geo_spectral_peaks_{day}_window{window_length:.0f}_overlap{overlap:.1f}_prom{prom_threshold:.0f}_rbw{rbw_threshold:.1f}.csv"
    inpath = join(indir, filename)
    peak_df = read_csv(inpath)
    
    # Group the detections by time and frequency
    peak_df["time_bin"] = Grouper(key="time", freq=f"{time_bin_width:.0f}T")
    
    num_freq_bins = int((maxfreq - minfreq) / freq_bin_width)
    freq_bin_edges = linspace(minfreq, maxfreq, num_freq_bins + 1)
    peak_df["freq_bin"] = cut(peak_df["frequency"], freq_bin_edges, include_lowest=True, right=False)

    grouped = peak_df.groupby(['time_bin', 'freq_bin']).size().unstack(fill_value=0)
    print(grouped)

    